In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import polars as pl
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features
from src.visualization import visualize

sns.set_theme(palette="pastel")
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", None)

# Asumiendo que es una ocupacion del 80% de camas
INDICE_OCUPACION_CAMAS = 1.25

# Agrega las columnas de poblacion de interes
COLUMNAS_POBLACION_INTERES = {str(i): i for i in range(2017, 2036)}

# Proyección de casos INT al 2035

In [3]:
# Lee los casos de todos los macroprocesos
RUTA_ARCHIVOS = "../data/interim/casos_teoricos_diagnosticos.xlsx"
_, casos_macroproceso = build_features.leer_casos_macroprocesos(RUTA_ARCHIVOS)

# Obtiene los casos hospitalizados
casos_hospitalizados = casos_macroproceso.query("tipo_paciente == 'hospitalizados'")

In [4]:
# Los diagnosticos que tienen un porcentaje de hosp. mayor a 0
diagnosticos_relevantes = list(casos_hospitalizados.index)

# Ademas, los diagnosticos que estan agrupados dentro de los que aportan a AC
# Solo deja los diagnosticos que esten agrupados
diagnosticos_a_reasignar = casos_hospitalizados.dropna(subset="Diagnosticos Contenidos")

# Separa la lista de diagnosticos en filas
diagnosticos_agrupados = diagnosticos_a_reasignar.explode("Diagnosticos Contenidos")
diagnosticos_agrupados = list(diagnosticos_agrupados["Diagnosticos Contenidos"].unique())

# Suma los diagnosticos relevantes y los agrupados
diagnosticos_a_buscar_en_egresos = sorted(diagnosticos_relevantes + diagnosticos_agrupados)

## Obtención de casos atendidos entre 2017 y 2020 para diagnósticos relevantes.

Estos se obtendrán del análisis de egresos hospitalarios a nivel país DEIS.

In [5]:
# Lee los egresos del Torax
egresos_torax = pl.read_csv("../data/raw/6_egresos_deis/egresos_procesados_112103.csv")

En primer lugar, se filtrarán ambas bases de datos para solamente tener la información de los diagnósticos más relevantes para el INT. Luego de esto, se calculará la cantidad de egresos, dias de estada y pacientes para cada uno de los diagnósticos. Estos insumos serán utilizados para estimar la demanda.


In [6]:
# Filtra solamente los diagnosticos mas relevantes del Torax
egresos_mas_relevantes_torax = egresos_torax.filter(
    (pl.col("DIAG1").is_in(diagnosticos_a_buscar_en_egresos))
)

## Reasginación de diagnósticos y agrupación

En este apartado se reasignarán diagnósticos de la base de datos DEIS a un grupo mayor. Por ejemplo,
los siguientes diagnósticos:

- C341 - Tumor maligno del lóbulo superior, bronquio o pulmón
- C342 - Tumor maligno del lóbulo medio, bronquio o pulmón
- C343 - Tumor maligno del lóbulo inferior, bronquio o pulmón
- C780 - Tumor maligno secundario del pulmón
- C782 - Tumor maligno secundario de la pleura
- D381 - Tumor de comportamiento incierto o desconocido de la tráquea, de los bronquios y del pulmón

Todos estos se reasignarán al diagnóstico C34XN (N, para que no se mezclen con los C34X asignados realmente así). Esto, con el fin de poder calcular las estadísticas agrupadas para todos estos diagnósticos (egresos por persona, días de estada).

In [7]:
for row in diagnosticos_a_reasignar.itertuples():
    diagnostico_nuevo = row[0]
    diagnosticos_antiguos = row[1]

    print(f"Cambiando {diagnosticos_antiguos} a {diagnostico_nuevo}")
    egresos_mas_relevantes_torax = egresos_mas_relevantes_torax.with_columns(
        pl.col("DIAG1").replace(diagnosticos_antiguos, diagnostico_nuevo)
    )

Cambiando ['C341', 'C342', 'C343', 'C780', 'C782', 'D381'] a C34N
Cambiando ['I251'] a Enfermedad Coronaria
Cambiando ['I269'] a Enfermedad tromboembólica crónica pulmonar
Cambiando ['I052', 'I080', 'I081'] a I051
Cambiando ['I352'] a I350
Cambiando ['Q201', 'Q202', 'Q203', 'Q204', 'Q205', 'Q206', 'Q208', 'Q209', 'Q210', 'Q211', 'Q212', 'Q213', 'Q214', 'Q218', 'Q220', 'Q221', 'Q222', 'Q223', 'Q224', 'Q225', 'Q228', 'Q230', 'Q231', 'Q233', 'Q240', 'Q241', 'Q244', 'Q245', 'Q246', 'Q248', 'Q249', 'Q250', 'Q251', 'Q253', 'Q254', 'Q255', 'Q256', 'Q257', 'Q258', 'Q259', 'Q264', 'Q268', 'Q272', 'Q273', 'Q288', 'Q289', 'Q311', 'Q320', 'Q321', 'Q330', 'Q331', 'Q332', 'Q334', 'Q338', 'Q341', 'Q348', 'Q678', 'Q765', 'Q766', 'Q767', 'Q768', 'Q769', 'Q780', 'Q790', 'Q798', 'Q839', 'Q850', 'Q858', 'Q859', 'Q874', 'Q893'] a QXXX


In [8]:
# Obtiene resumen de metricas para el Torax
metricas_diags_relevantes_torax = (
    build_features.calcular_resumen_metricas_desagregadas_y_agrupadas_en_anios(
        egresos_mas_relevantes_torax, 2017, 2019
    )
)

In [9]:
# Define las columnas que se quieran corregir
columna_dias_estada = ("agrupado_entre_2017_2019", "dias_estada_promedio")

# Corrige dias de estada de estenosis e insuficiencia aortica
metricas_diags_relevantes_torax.loc["C34N", columna_dias_estada] = 10
metricas_diags_relevantes_torax.loc["I350", columna_dias_estada] = 10
metricas_diags_relevantes_torax.loc["I351", columna_dias_estada] = 10

# Corrige dias de estada de trasplante cardiaco y pulmonar - ACUMULADO GRD entre 2019 y 2022
metricas_diags_relevantes_torax.loc["Z941", columna_dias_estada] = 65.8065
metricas_diags_relevantes_torax.loc["Z942", columna_dias_estada] = 93.7037

## Estimación de casos Hospitalizados utilizando % de Hospitalización del INT

En este caso, se utilizará el % de hospitalización del INT de los pacientes ambulatorios.

In [10]:
# Se obtiene la cantidad de egresos que debiese tener el INT, asumiendo que trabajara con la misma
# eficiencia
egresos_estimados_INT = casos_hospitalizados[COLUMNAS_POBLACION_INTERES.keys()].mul(
    metricas_diags_relevantes_torax[("agrupado_entre_2017_2019", "egresos_por_paciente")], axis=0
)

# Se obtiene la cantidad de dias de estada que debiese el INT, asumiendo que trabajara con la misma
# eficiencia
dias_estada_estimados_int = egresos_estimados_INT.mul(
    metricas_diags_relevantes_torax[("agrupado_entre_2017_2019", "dias_estada_promedio")], axis=0
)

# Estima la cantidad de camas necesaarias por diagnostico
camas_estimadas_int = (dias_estada_estimados_int / 365.25) * INDICE_OCUPACION_CAMAS

# Calcula las camas totales necesarias
camas_totales_int_porcentaje_amb = camas_estimadas_int.sum()

In [11]:
# Se quiere tener un 60% de UPC (1:2, UCI:UTI, y 40% de Medias)
FRACCION_UCI = 0.21
FRACCION_UTI = 0.42
FRACCION_MEDIAS = 0.37

# Agrega las camas necesarias para trasplantes a las del 2035
camas_al_2035 = (camas_totales_int_porcentaje_amb["2035"])
# Obtiene la cantidad de camas UCI, UTI y Medias sin redondear
camas_uci = (camas_al_2035 * FRACCION_UCI)
camas_uti = (camas_al_2035 * FRACCION_UTI)
camas_medias = (camas_al_2035 * FRACCION_MEDIAS)

# Obtiene la cantidad de camas UCI, UTI y Medias redondeadas
camas_uci_redondeadas = round(camas_al_2035 * FRACCION_UCI)
camas_uti_redondeadas = round(camas_al_2035 * FRACCION_UTI)
camas_medias_redondeadas = round(camas_al_2035 * FRACCION_MEDIAS)
camas_al_2035_redondeadas = round(camas_al_2035)

In [12]:
print(f"> Camas + Trasplantes 2035: {camas_al_2035:.2f}")
print(f"> UCI: {camas_uci:.2f} camas")
print(f"> UTI: {camas_uti:.2f} camas")
print(f"> Medias: {camas_medias:.2f} camas")
print()
print(f"> UCI Redondeadas: {camas_uci_redondeadas} camas")
print(f"> UTI Redondeadas: {camas_uti_redondeadas} camas")
print(f"> Medias Redondeadas: {camas_medias_redondeadas} camas")
print(f"> Total Redondeadas: {camas_al_2035_redondeadas} camas")

> Camas + Trasplantes 2035: 230.55
> UCI: 48.42 camas
> UTI: 96.83 camas
> Medias: 85.30 camas

> UCI Redondeadas: 48 camas
> UTI Redondeadas: 97 camas
> Medias Redondeadas: 85 camas
> Total Redondeadas: 231 camas


## Obtener resumen para MINSAL

In [13]:
# Obtiene porcentaje de hospitalizados
porcentaje_hospitalizados = pd.read_excel(
    "../data/raw/3_incidencias_y_porcentajes_marcoprocesos/incidencias_y_prevalencias_INT.xlsx"
)

porcentaje_hospitalizados["Diagnostico"] = (
    porcentaje_hospitalizados["Diagnostico"].str.split(" - ").str[0]
)

# Filtra los porcentajes de hospitalizados
porcentaje_hospitalizados = porcentaje_hospitalizados.set_index("Diagnostico")
porcentaje_hospitalizados = porcentaje_hospitalizados["Porcentaje Pacientes Hospitalizados"]
porcentaje_hospitalizados = porcentaje_hospitalizados.dropna()
porcentaje_hospitalizados = porcentaje_hospitalizados[porcentaje_hospitalizados > 0]

In [14]:
# Obtiene resumen para MINSAL
resumen_hosp = pd.DataFrame(
    {
        "porcentaje_hospitalizado": porcentaje_hospitalizados,
        "casos_hospitalizados_2035": casos_hospitalizados["2035"],
        "egresos_por_paciente_agrupado_2017_a_2019": metricas_diags_relevantes_torax[
            ("agrupado_entre_2017_2019", "egresos_por_paciente")
        ],
        "egresos_2035": egresos_estimados_INT["2035"],
        "dias_estada_promedio_agrupado_2017_a_2019": metricas_diags_relevantes_torax[
            ("agrupado_entre_2017_2019", "dias_estada_promedio")
        ],
        "dias_estada_totales_2035": dias_estada_estimados_int["2035"],
        "camas_totales_2035": camas_estimadas_int["2035"],
    }
)

Finalmente, une el resumen de casos teoricos desde el cuadernillo 1.0 con el resumen de este cuadernillo. Esto obtiene la tabla final para MINSAL.

In [15]:
resumen_casos_teoricos = pd.read_excel(
    "../data/interim/casos_teoricos_diagnosticos.xlsx", sheet_name="resumen_total_INT"
)

resumen_casos_teoricos["Diagnostico_codigo"] = (
    resumen_casos_teoricos["Diagnostico"].str.split(" - ").str[0]
)
resumen_casos_teoricos = resumen_casos_teoricos.set_index("Diagnostico_codigo")

resumen_minsal = resumen_casos_teoricos.merge(
    resumen_hosp, how="right", left_index=True, right_index=True
)

resumen_minsal = resumen_minsal.reset_index(drop=True).set_index("Diagnostico")

## Guardar archivos

In [16]:
archivos_a_guardar = {
    "resumen_MINSAL": resumen_minsal,
    "metricas_relevantes_INT": metricas_diags_relevantes_torax,
    "casos_hospitalizados_INT": casos_hospitalizados,
    "egresos_estimados_INT": egresos_estimados_INT,
    "dias_estada_estimados_INT": dias_estada_estimados_int,
    "camas_estimadas_desglosadas_INT": camas_estimadas_int,
    "camas_totales_INT": camas_totales_int_porcentaje_amb,
    "resumen_total_hosp_INT": resumen_hosp,
}

with pd.ExcelWriter("../data/interim/1.0_estimacion_atencion_cerrada_INT.xlsx") as file:
    for nombre_hoja, df_a_guardar in archivos_a_guardar.items():
        print(f"Guardando {nombre_hoja}")
        df_a_guardar.to_excel(file, sheet_name=nombre_hoja)

Guardando resumen_MINSAL
Guardando metricas_relevantes_INT
Guardando casos_hospitalizados_INT
Guardando egresos_estimados_INT
Guardando dias_estada_estimados_INT
Guardando camas_estimadas_desglosadas_INT
Guardando camas_totales_INT
Guardando resumen_total_hosp_INT
